In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load your dataset
data = pd.read_csv("train.csv") 

# Drop columns we won't use (e.g., id, CustomerId, Surname)
data = data.drop(columns=['id', 'CustomerId', 'Surname'])

# Encode categorical features
le = LabelEncoder()
data['Geography'] = le.fit_transform(data['Geography'])  # e.g., France=0, Spain=1, Germany=2
data['Gender'] = le.fit_transform(data['Gender'])        # Male=1, Female=0

# Separate features and target
X = data.drop(columns=['Exited']).values
y = data['Exited'].values

# Normalize numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Add bias term
X = np.hstack((np.ones((X.shape[0], 1)), X))  # shape: (n_samples, n_features+1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def predict(X, weights):
    return (sigmoid(np.dot(X, weights)) >= 0.5).astype(int)


In [9]:
def logistic_regression_gradient_ascent(X, y, lr=0.1, epochs=1000):
    m, n = X.shape
    weights = np.zeros(n)

    for _ in range(epochs):
        z = np.dot(X, weights)
        predictions = sigmoid(z)
        gradient = np.dot(X.T, y - predictions)
        weights += lr * gradient

    return weights


In [10]:
def perceptron(X, y, epochs=100):
    m, n = X.shape
    weights = np.zeros(n)
    y_ = np.where(y == 0, -1, 1)

    for _ in range(epochs):
        for i in range(m):
            if y_[i] * np.dot(X[i], weights) <= 0:
                weights += y_[i] * X[i]

    return weights


In [11]:
def logistic_regression_newton(X, y, epochs=10):
    m, n = X.shape
    weights = np.zeros(n)

    for _ in range(epochs):
        z = np.dot(X, weights)
        h = sigmoid(z)
        gradient = np.dot(X.T, y - h)
        diag = h * (1 - h)
        H = -np.dot(X.T, diag[:, np.newaxis] * X)
        weights -= np.linalg.pinv(H).dot(gradient)  # Use pseudo-inverse to avoid singular errors

    return weights


In [12]:
# Train models
weights_ga = logistic_regression_gradient_ascent(X_train, y_train)
weights_perceptron = perceptron(X_train, y_train)
weights_newton = logistic_regression_newton(X_train, y_train)

# Predict
preds_ga = predict(X_test, weights_ga)
preds_perceptron = predict(X_test, weights_perceptron)
preds_newton = predict(X_test, weights_newton)

# Evaluate
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

print("Gradient Ascent Accuracy:", accuracy(y_test, preds_ga))
print("Perceptron Accuracy:", accuracy(y_test, preds_perceptron))
print("Newton’s Method Accuracy:", accuracy(y_test, preds_newton))


C:\Users\InaequoSolutions-PC\AppData\Local\Temp\ipykernel_23312\4236883662.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Gradient Ascent Accuracy: 0.6341382131063108
Perceptron Accuracy: 0.7662617020631988
Newton’s Method Accuracy: 0.8270063925833914
